Read arxiv data base of scientific articles.

In [4]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-f260c7dd-ea22-450d-ac4f-411d7bbe11a8',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'fN2y36SJQsBNLN7BhTsrFNjuEIaYuqSsJn5059Il-meW'
}

configuration_name = 'os_e245254457804a5abc5f0819c8c11f69_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json


df_arxiv_data = spark.read.json(cos.url('arxivData_min.json', 'advanceddatasciencecapstone-donotdelete-pr-eegj6wm2sruxqi'))

[Row(author="[{'name': 'Ahmed Osman'}, {'name': 'Wojciech Samek'}]", day=1, id='1802.00209v1', link="[{'rel': 'alternate', 'href': 'http://arxiv.org/abs/1802.00209v1', 'type': 'text/html'}, {'rel': 'related', 'href': 'http://arxiv.org/pdf/1802.00209v1', 'type': 'application/pdf', 'title': 'pdf'}]", month=2, summary='We propose an architecture for VQA which utilizes recurrent layers to\ngenerate visual and textual attention. The memory characteristic of the\nproposed recurrent attention units offers a rich joint embedding of visual and\ntextual features and enables the model to reason relations between several\nparts of the image and question. Our single model outperforms the first place\nwinner on the VQA 1.0 dataset, performs within margin to the current\nstate-of-the-art ensemble model. We also experiment with replacing attention\nmechanisms in other state-of-the-art models with our implementation and show\nincreased accuracy. In both cases, our recurrent attention mechanism improves

In [5]:
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import regexp_replace, col

df_arxiv_data_tokenized = df_arxiv_data.withColumn("title", regexp_replace(col("title"), '\\n', ' '))

regexTokenizer = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'title', outputCol = 'title_tkn')
df_arxiv_data_tokenized = regexTokenizer.transform(df_arxiv_data_tokenized)
df_arxiv_data_tokenized.show(3)

swr = StopWordsRemover(inputCol = 'title_tkn', outputCol = 'title_sw_removed')
all_stop_words = swr.getStopWords()
all_stop_words.extend(['learn', 'learning', 'performance', 'approach', 'result', 'results'])
swr.setStopWords(all_stop_words)
df_arxiv_data_swr = swr.transform(df_arxiv_data_tokenized)
df_arxiv_data_swr.show(3)


+--------------------+---+------------+--------------------+-----+--------------------+--------------------+--------------------+----+--------------------+
|              author|day|          id|                link|month|             summary|                 tag|               title|year|           title_tkn|
+--------------------+---+------------+--------------------+-----+--------------------+--------------------+--------------------+----+--------------------+
|[{'name': 'Ahmed ...|  1|1802.00209v1|[{'rel': 'alterna...|    2|We propose an arc...|[{'term': 'cs.AI'...|Dual Recurrent At...|2018|[dual, recurrent,...|
|[{'name': 'Ji You...| 12|1603.03827v1|[{'rel': 'alterna...|    3|Recent approaches...|[{'term': 'cs.CL'...|Sequential Short-...|2016|[sequential, shor...|
|[{'name': 'Iulian...|  2|1606.00776v2|[{'rel': 'alterna...|    6|We introduce the ...|[{'term': 'cs.CL'...|Multiresolution R...|2016|[multiresolution,...|
+--------------------+---+------------+--------------------+----

In [6]:
from pyspark.ml.feature import IDF, HashingTF
hashingTF = HashingTF(inputCol='title_sw_removed', outputCol='title_hash')
tf = hashingTF.transform(df_arxiv_data_swr)

idf = IDF(inputCol='title_hash', outputCol='title_vec')
tf.cache()
fit_idf = idf.fit(tf)

In [7]:
df_arxiv_data_vec = fit_idf.transform(tf)

In [11]:
df_arxiv_data_vec.write.parquet(cos.url('arxiv_data_transformed.parquet', 'advanceddatasciencecapstone-donotdelete-pr-eegj6wm2sruxqi'))